# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    
    df = pd.DataFrame()
    
    import os
    
    for filename in os.scandir('names'):
        if (filename.is_file()) & (filename.path.endswith('txt')):
            df_fichier = pd.read_csv(filename.path, header=None).rename(columns={0:'name' , 1:'gender' , 2:'births'})
            annee = filename.path[-8:-4]
            df_fichier['year'] = int(annee)
            df_fichier = df_fichier.reindex(columns=['year', 'name', 'gender', 'births'])
            df = pd.concat([df, df_fichier]).reset_index(drop=True)
    
    df['year'] = df['year'].astype('uint16')
    df['births'] = df['births'].astype('uint32')

    df = df.sort_values(['year', 'gender', 'births', 'name'], ascending = [True, True, False, True]).reset_index(drop=True)
    
    df = df.drop_duplicates().reset_index(drop=True)
        
    return df

In [3]:
df_us = df_names_us()
df_us

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


In [4]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2052781 entries, 0 to 2052780
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   year    uint16
 1   name    object
 2   gender  object
 3   births  uint32
dtypes: object(2), uint16(1), uint32(1)
memory usage: 43.1+ MB


In [5]:
df_us['gender'].isin(['F','M']).all()

True

In [6]:
df_us.duplicated().value_counts()
# ==> Pas de lignes dupliquées

False    2052781
dtype: int64

In [7]:
len(df_us.loc[df_us.isna().any(axis=1)])
# ==> Pas de ligne avec au moins un NaN

0

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [8]:
# names fr
def df_names_fr():
    
    import re
    
    df = pd.read_csv('nat2021.csv', sep=';', header=None, skiprows=1, names=['gender', 'name', 'year', 'births'], keep_default_na=False)
    df = df.reindex(columns=['year', 'name', 'gender', 'births'])
    
    df = df.drop_duplicates().reset_index(drop=True)
    
    # Traitement de la colonne 'gender'
    df['gender'] = df['gender'].map({2: 'F', 1: 'M'})
    df = df.drop(df[~df['gender'].isin(['F','M'])].index).reset_index(drop=True)
    
    # Traitement de la colonne 'year'
    df['year'] = df['year'].str.strip()
    df = df.drop(df[~df['year'].str.isnumeric()].index).reset_index(drop=True)
    df['year'] = df['year'].astype('uint16')
    df = df.loc[df['year'].between(1900,2021)].reset_index(drop=True)
    
    # Traitement de la colonne 'births'
    df['births'] = df['births'].astype('uint32')
    
    # Traitement de la colonne 'name'
    df['name'] = df['name'].str.strip()
    df['name'] = df['name'].str.title()
    a = '[A-ZŠŽŸÀ-ÖØ-Þ]'
    b = '[a-zšœžß-öø-ÿ]'
    ER = "(({}{}+)|(({}{}*)(\'({}{}*))+))(\-(({}{}*)|(({}{}*)(\'({}{}*))+)))*".format(a,b,a,b,a,b,a,b,a,b,a,b)
    df = df.loc[df['name'].apply(lambda x: bool(re.fullmatch(ER, x)))].reset_index(drop=True)
    
    # Tri des données
    df = df.sort_values(['year', 'gender', 'births', 'name'], ascending = [True, True, False, True]).reset_index(drop=True)
    
    return df

In [9]:
df_fr = df_names_fr()
df_fr

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
648325,2021,Élyam,M,3
648326,2021,Énael,M,3
648327,2021,Éros,M,3
648328,2021,Étan,M,3


In [10]:
df_fr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648330 entries, 0 to 648329
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   year    648330 non-null  uint16
 1   name    648330 non-null  object
 2   gender  648330 non-null  object
 3   births  648330 non-null  uint32
dtypes: object(2), uint16(1), uint32(1)
memory usage: 13.6+ MB


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [11]:
# taux de change
def df_taux_change(devises):
    
    df = pd.read_csv('Webstat_Export.csv', sep=';')

    import re
    
    df = df.drop(index=[0,2,3,4])
    df = df.set_index(df.columns[0])
    df = df.loc[:,df.iloc[0,:].apply(lambda x: bool(re.fullmatch('.*\([A-Z]{3}\)', x)))]
    df.iloc[0,:] = df.iloc[0,:].apply(lambda x: x[-4:-1])
    df.columns = df.iloc[0,:]
    df = df.iloc[1:,:]
    df.index.name = 'Date'
    df.columns.name = 'ISO3'
    df.index = pd.to_datetime(df.index, format="%d/%m/%Y")
    df = df.sort_index(ascending=True)

    df = df.apply(lambda x: x.str.replace(',', '.'))
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce') )
    df = df.astype('float')
    
    df = df.loc[:,devises]
    df= df.dropna()
    
    return df

In [12]:
df_taux_change(["CHF", "GBP", "USD"])

ISO3,CHF,GBP,USD
Date,,,
1999-01-04,1.6168,0.71110,1.1789
1999-01-05,1.6123,0.71220,1.1790
1999-01-06,1.6116,0.70760,1.1743
1999-01-07,1.6165,0.70585,1.1632
1999-01-08,1.6138,0.70940,1.1659
...,...,...,...
2022-09-14,0.9612,0.86498,0.9990
2022-09-15,0.9572,0.86934,0.9992
2022-09-16,0.9579,0.87400,0.9954


### Tests

In [13]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [14]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [15]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 18.269s

OK
